In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
%sendtofile --source wifimqttconnections.txt config.txt
%sendtofile --append config.txt

pinled       -2

Sent 1 lines (15 bytes) to config.txt.


In [3]:
%sendtofile --source ../Sensor_Kennel/SI7021_funcs.py
%sendtofile --source ../Sensor_Kennel/SHT31D_funcs.py

Sent 43 lines (1090 bytes) to SHT31D_funcs.py.


In [4]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 81 lines (2677 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [5]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [6]:
%sendtofile main.py

import time
from machine import Pin
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask, flashledconnectedtask
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count
from mqtt_as import MQTTClient
from SI7021_funcs import setupSI7021, SI7021checkchip, SI7021printstatus, SI7021humiditytemp
from SHT31D_funcs import SHT31Dsetup, readSHT31D

import network
network.WLAN().active(0)  # disable the connection at startup
flashpinled(5, 300, 300)

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
print(list(map(hex, i2c.scan())))

topicfan = topicstem+"/fan"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"
topicSI7021 = topicstem+'/si7021'
topicSHT31D = topicstem+'/sht31d'

async def si7021task():
    btoinit = True
    for i in itertools_count():
        try:
            if btoinit:
                setupSI7021(i2c)
                btoinit = False
                await asyncio.sleep_ms(100)
            h, t = SI7021humiditytemp()
            payload = "%f %f"%(h, t)
            if client.isconnected():
                print(payload)
                await client.publish(topicSI7021, payload)
        except OSError as e:
            print("si7021task", e)
            btoinit = True
        await asyncio.sleep_ms(500)

async def sht31dtask():
    btoinit = True
    for i in itertools_count():
        try:
            if btoinit:
                SHT31Dsetup(i2c)
                btoinit = False
                await asyncio.sleep_ms(350)
            t, h = readSHT31D()
            payload = "%f %f"%(h, t)
            if client.isconnected():
                print(payload)
                await client.publish(topicSHT31D, payload)
        except OSError as e:
            print("sht31dtask", e)
            btoinit = True
        await asyncio.sleep_ms(500)
        
    
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber, "to", client._ssid, client.server)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['port'] = 4500
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
    
aloop = asyncio.get_event_loop()
aloop.create_task(si7021task())
aloop.create_task(sht31dtask())
aloop.create_task(flashledconnectedtask(client))
aloop.create_task(mqttconnecttask(client))

aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))



Sent 92 lines (3008 bytes) to main.py.


In [11]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [12]:
from network import WLAN
WLAN().active(0)

[leftinbuffer] ['scandone']
[leftinbuffer] ['no JulianS9 found, reconnect after 1s']
[leftinbuffer] ['reconnect']
[leftinbuffer] ['scandone']
[leftinbuffer] ['no JulianS9 found, reconnect after 1s']
[leftinbuffer] ['reconnect']
[leftinbuffer] ['scandone']
[leftinbuffer] ['no JulianS9 found, reconnect after 1s']
[leftinbuffer] ['reconnect']
[leftinbuffer] ['scandone']
[leftinbuffer] ['no JulianS9 found, reconnect after 1s']
[leftinbuffer] ['reconnect']
[leftinbuffer] ['scandone']
[leftinbuffer] ['no JulianS9 found, reconnect after 1s']
[leftinbuffer] ['reconnect']
[leftinbuffer] ['scandone']
[leftinbuffer] ['no JulianS9 found, reconnect after 1s']
[leftinbuffer] ['reconnect']
scandone
del if0
mode : softAP(3e:71:bf:2c:af:0f)


In [69]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [70]:
import machine
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
print(list(map(hex, i2c.scan())))


['0x40', '0x44']


In [72]:


from SHT31D_funcs import SHT31Dsetup, readSHT31D
x = SHT31Dsetup(i2c)
print(readSHT31D())
    
    

b'\x80\x10\xe1' 0xe1
b'\x80pZ' 0x5a
(16.6846, 52.9625)
